In [1]:
using Random
#using Printf
using LinearAlgebra
using Statistics

In [2]:
function asian_barrier_option(T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS)
    tmp1 = mu * T / N_STEPS
    tmp2 = exp(-r * T)
    tmp3 = sqrt(T / N_STEPS)
    d_s = zeros(Float64, N_PATHS)
    
    for i in 1:N_PATHS
        s_curr = S0
        running_average = 0.0
        
        for n in 1:N_STEPS
            # Update the current price
            s_curr += tmp1 * s_curr + sigma * s_curr * tmp3 * d_normals[i + (n - 1) * N_PATHS]
            
            # Update the running average
            running_average += (s_curr - running_average) / n
            
            # Check the barrier
            if running_average <= B
                break
            end
        end
        
        # Calculate payoff
        payoff = max(running_average - K, 0)
        
        # Discounted payoff
        d_s[i] = tmp2 * payoff
    end

    return mean(d_s), 1.96 * std(d_s)/sqrt(N_PATHS)
end


asian_barrier_option (generic function with 1 method)

In [3]:
# Example usage
N_STEPS = 365
N_PATHS = 1000000
T = 1.0
K = 110.0
B = 100.0
S0 = 120.0
sigma = 0.35
mu = 0.1
r = 0.05
d_normals = randn(Float64, N_STEPS * N_PATHS)  # Standard normal random numbers

mean_ds, err = asian_barrier_option(T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS)

(18.688704378890158, 0.04161837900794215)

In [5]:
@time asian_barrier_option(T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS)

  1.871627 seconds (4 allocations: 7.629 MiB)


(18.688704378890158, 0.04161837900794215)

In [22]:
asian_barrier_option(T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS)

(18.746854091771066, 0.04169004063779831)

In [ ]:
# 株価とインプライド・ボラティリティの範囲ごとにアジアンバリアオプション価格の行列を計算する関数
function calculate_asian_barrier_option_matrix(K, B, T, mu, r, N_STEPS, N_PATHS)
    S0_values = range(90, stop=120, length=101)[1:end-1]
    implied_vol_values = range(0.15, stop=0.25, length=101)[1:end-1]
    price_matrix = Array{Float64}(undef, length(S0_values), length(implied_vol_values))
    err = Array{Float64}(undef, length(S0_values), length(implied_vol_values))
    d_normals = randn(Float64, N_STEPS * N_PATHS)  # Standard normal random numbers
    for i in 1:length(S0_values)
        for j in 1:length(implied_vol_values)
            S0 = S0_values[i]
            sigma = implied_vol_values[j]
            #price_matrix[i, j] = asian_barrier_option(S0, K, B, T, sigma, mu, r, N_STEPS, N_PATHS)
            price_matrix[i, j], err[i, j] = asian_barrier_option(T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS)
        end
    end

    return price_matrix, err
end

# 例としての利用
K = 110                # 行使価格
B = 100                 # バリア価格
T = 1.0                # 満期までの期間（1年）
mu = 0.1              # ドリフト（株価の期待成長率）
r = 0.05               # 無リスク金利
N_STEPS = 365          # タイムステップ数
N_PATHS = 1000000         # モンテカルロパス数
price_matrix, err = calculate_asian_barrier_option_matrix(K, B, T, mu, r, N_STEPS, N_PATHS)

# 結果の行列とパラメータの範囲を表示
println("アジアンバリアオプション価格の行列:")
display(price_matrix)
#println("株価 S0 の範囲: ", S0_values)
#println("インプライド・ボラティリティの範囲: ", implied_vol_values)

In [ ]:
using JLD2
JLD2.@save "price_matrix_asianbarrier_N_PATHS$N_PATHS.jld2" price_matrix

In [17]:
mean(vec(err))